In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

Notebook settings

In [ ]:
# Automatically reloads any modules that are imported, 
# so that any changes made to the module files are reflected # without needing to restart the Jupyter kernel.
# load autoreload module
%load_ext autoreload
# mode 1 reloads only when an import statement is called. For production
# mode 2 reloads before execution of every cell
%autoreload 2

# limit the number of rows that are shown with printing dataframes
pd.set_option('display.max_rows', 5)

In [ ]:
df = pd.read_csv("data/cleaned/pubs.csv")

### Filter for biomedical institutions

In [ ]:
biomedical = ['amcpub', 'lumc', 'vumc', 'umcu'] 
subset = df[df['institution'].isin(biomedical)]
subset = subset.reset_index(drop=True)

### Datetime format for `year`

In [ ]:
# datetime change, run once
for index, value in enumerate(subset['year']):
    subset.at[index, 'year'] = datetime.strptime(str(value), '%Y')
date_5_years_ago = datetime.now() - timedelta(days=5*365)
new_df = subset[subset['year'] >= date_5_years_ago]
new_df = new_df.reset_index(drop=True)

### Additional cleaning

In [ ]:
for column in new_df.columns:
    for index, value in new_df[column].items():
        if str(value).startswith(','):
            new_df.at[index, column] = np.nan
        if str(value).startswith('Surgery'):             # from manual check 
            new_df.at[index, column] = np.nan

In [ ]:
nan_rows = new_df.iloc[:, 7:17].isnull().all(axis=1)
new_df = new_df[~nan_rows]
new_df.reset_index(drop=True, inplace=True)


In [ ]:
for index, row in new_df.iterrows():
    for i in range(1, 10):
        current_col = 'contributor_' + str(i)
        next_col = 'contributor_' + str(i + 1)
        
        # Check if current column is NaN and next column is not NaN
        if pd.isna(row[current_col]) and not pd.isna(row[next_col]):
            print(row)
            # Replace current column with next column and set next column to NaN
            new_df.at[index, current_col] = row[next_col]
            new_df.at[index, next_col] = np.nan

In [ ]:
df = new_df
df = df.dropna(subset=['phd_name', 'title', 'contributor_1'])
df.reset_index(drop=True, inplace=True)
df

In [ ]:
df = df.drop_duplicates(subset=['phd_name'], keep='last')
df.reset_index(drop=True, inplace=True)
df

### Save to CSV - USE IN `Open_Alex_Final.ipynb`

In [ ]:
df.to_csv('data/cleaned/biomedical_data.csv', index=False)